In [1]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="get_location")

location = "Morongo Valley"
location = geolocator.geocode(location)
latitude = location.latitude
longitude = location.longitude

print("Latitude:", latitude)
print("Longitude:", longitude)

Latitude: 34.0537585
Longitude: -116.59672363338927


In [2]:
def get_coordinate(query: str):
    """
    Get the latitude and longitude from an input query string
    """
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="get_location")
 
    location = geolocator.geocode(query)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
        return latitude, longitude
    else:
        return None

def partition_by_caps(input):
    import re
    parts = re.findall('[A-Z][^A-Z]*', input)
    return parts

In [3]:
import pandas as pd
df = pd.read_csv('communities.csv', index_col=None, header=None)
types = ['city', 'township', 'town', 'borough', 'village', 'division', 'district', 'Valle']
df[df[3].str[-4:]=='alle'][3]

1655    TwentyninePalms-MorongoValle
Name: 3, dtype: object

In [4]:
sep = '\s*,\s*'

# Read CSV file using custom separator
state_code_df = pd.read_csv('us-state-code.csv', sep=sep, engine='python')
def get_state_name(code:int):
    return state_code_df.query('st=='+str(code))['stname'].to_string(index=False)

get_state_name(42)

'Pennsylvania'

In [5]:
df['state_name'] = df[0].apply(get_state_name)

In [6]:
city_lst = df[3].apply(partition_by_caps).to_list()

In [7]:
type_list = []
for city in city_lst:
    for t in types:
        if t in city[-1]:
            city[-1] = city[-1].removesuffix(t)
            type_list.append(t)
            removed = True
            break
        


    
df['type'] = type_list
df['city'] = city_lst

In [8]:
query_strs = []

for i in range(len(df['type'])):
    query_msg = ''
    for name in df['city'][i]:
        query_msg += name + ' '
    query_msg = query_msg[:-1]
    query_msg += ', '+df['state_name'][i]
    
    query_strs.append(query_msg)

In [9]:
# this will take ~ 20 min
latitudes = []
longitudes = []
for i in range(len(df['type'])):
    try:
        coor = get_coordinate(query_strs[i])
        if coor is not None:
            latitudes.append(coor[0])
            longitudes.append(coor[1])
        else:
            latitudes.append(None)
            longitudes.append(None)
    except:
        latitudes.append(None)
        longitudes.append(None)

In [ ]:
pd.DataFrame({'latitude':latitudes,'longitude':longitudes}).to_csv('coordinates.csv')

In [ ]:
# pd.read_csv('coordinates.csv')[['latitude', 'longitude']].to_csv('coordinates.csv')

In [ ]:
coor = pd.read_csv('coordinates.csv')

In [ ]:
coor.isna().sum()

Unnamed: 0     0
latitude      12
longitude     12
dtype: int64